In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# import notebooks
import import_ipynb
from bg_remover_u2net import load_u2net, predict_mask, apply_mask
from bg_generator_sd import load_sd_controlnet, generate_background

In [ ]:
device = 'cpu'
u2net_path = '../u2net.pth'

load models

In [ ]:
pipe = load_sd_controlnet(device)
u2net = load_u2net(u2net_path)

input

In [ ]:
image_path = '../images/img1.jpg'
prompt = "art mueseum"

remove background

In [ ]:
mask, image = predict_mask(image_path, u2net)
subject = apply_mask(image, mask)

# original
plt.imshow(image)
plt.axis('off')
plt.show()

# subject only
plt.imshow(subject)
plt.axis('off')
plt.show()

generate background

In [ ]:
background = generate_background(prompt, image_path, pipe, u2net, 6, 20)

In [ ]:
background

blend foreground and background

In [ ]:
def blend(fg_rgba, bg_rgb, blur_amount=1):
    """
    fg_rgba: Foreground image with alpha channel (H, W, 4)
    bg_rgb: Background image without alpha (H, W, 3)
    """

    # Convert PIL images → NumPy for processing
    fg = np.array(fg_rgba).astype(float)
    bg = np.array(bg_rgb).astype(float)

    # Split channels
    alpha = fg[:, :, 3] / 255.0    # normalize 0–1
    alpha = alpha.astype(np.float32)

    # --- Soft Edge: Blur the alpha mask ---
    alpha = cv2.GaussianBlur(alpha, (blur_amount, blur_amount), 0)

    # Ensure sizes match
    alpha = alpha[:, :, None]  # make shape (H, W, 1)

    # Foreground RGB
    fg_rgb = fg[:, :, :3]

    # Blend: fg * alpha + bg * (1 - alpha)
    blended = fg_rgb * alpha + bg * (1 - alpha)
    blended = blended.astype(np.uint8)

    return Image.fromarray(blended)

In [ ]:
result = blend(subject, background)

final output

In [ ]:
result